# Assignment 4: Convolutional Neural Networks

In [1]:
import os
import sys
from typing import List, Literal, Tuple

import cv2
import matplotlib.pyplot as plt
import numpy as np
import numpy.typing as npt
import torch

PROJECT_DIR = '../..'
sys.path.insert(0, PROJECT_DIR)

from models.cnn import CNN, MultiLabelCNN
from models.knn import KNN
from models.MLP import get_activation, get_loss
from performance_measures import ClassificationMeasures, MultiLabelClassificationMeasures, \
                                                                                RegressionMeasures

## Classification using CNN

### Data Loading and Preprocessing

In [2]:
def load_mnist_data(dataset_path: str, split: Literal['train', 'val', 'test'], \
                    count_digits: bool = False) -> List[Tuple[npt.NDArray[np.uint8], str | int]]:
    """ Loads the images and labels for a split of the Multi MNIST (double_mnist) dataset. """

    # Image, label pairs
    X, y = [], []

    for descriptor in os.listdir(f'{dataset_path}/{split}'):

        # Label
        if count_digits:
            label = len(descriptor) if descriptor != '0' else 0
        else:
            label = descriptor

        # Image
        for image_path in os.listdir(f'{dataset_path}/{split}/{descriptor}'):
            image = cv2.imread(f'{dataset_path}/{split}/{descriptor}/{image_path}', 0)

            # Store image and label
            X.append(image)
            y.append(label)

    return X, y

def one_hot_encoding(labels: List[str]):
    """ Creates one hot encoded vectors for each label value. """

    out = []
    for label in labels:
        encoding = np.zeros(10)
        if label != '0':
            encoding[[int(c) for c in label]] = 1
        out.append(encoding)
    return out

In [20]:
class MultiMNISTDataset(torch.utils.data.Dataset):

    def __init__(self, dataset_path: str, split: Literal['train', 'val', 'test'], \
                                                                task: Literal['count', 'predict']):

        if task == 'count':
            self.images, self.labels = load_mnist_data(dataset_path, split, count_digits=True)
        else:
            self.images, self.labels = load_mnist_data(dataset_path, split, count_digits=False)
            self.labels = one_hot_encoding(self.labels)

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        return torch.Tensor(self.images[idx]), self.labels[idx]

In [22]:
train_set = MultiMNISTDataset(f'{PROJECT_DIR}/data/external/double_mnist', 'train', 'count')
val_set = MultiMNISTDataset(f'{PROJECT_DIR}/data/external/double_mnist', 'val', 'count')
test_set = MultiMNISTDataset(f'{PROJECT_DIR}/data/external/double_mnist', 'test', 'count')

### Hyperparameter Tuning

In [ ]:
# 1. Identify the key hyperparameters to tune, including learning rate, dropout rate, the number of convolutional layers and optimizer choice.
# 2. Plot training and validation loss graphs for at least 5 different combinations of hyperparameters for each of the classification and regression models.
# 3. Identify the best perfoming model and include relevant plots and analyses in the report. Report accuracies of both classification and regression CNN on validation and test set.

### Feature Map Visualization

In [ ]:
# Feature maps reveal which patterns and features the CNN is focusing on at various layers. To obtain feature maps, you can access the outputs after each block of [ convolution, activation function, and pooling layer ] during the forward pass of the model. Visualize feature maps of any three images.

## Multi Label Classification

### Data Loading and Preprocessing

In [ ]:
train_set = MultiMNISTDataset(f'{PROJECT_DIR}/data/external/double_mnist', 'train', 'predict')
val_set = MultiMNISTDataset(f'{PROJECT_DIR}/data/external/double_mnist', 'val', 'predict')
test_set = MultiMNISTDataset(f'{PROJECT_DIR}/data/external/double_mnist', 'test', 'predict')

### Hyperparameter Tuning

In [ ]:
# 1. Experiment with different values of hyperparameters such as learning rate, number of epochs, dropout rate, the number of convolutional layers and optimizer choice.
# 2. Plot training and validation loss graphs for 5 or more such combinations of hyperparameters.
# 3. Identify the best perfoming model and include relevant plots and analyses in the report. For this model, report both the exact match accuracy and hamming accuracy on train, val and test data.